In [4]:
import cv2 as cv
import torch
import numpy as np

model_weights_path = './trained_model/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_weights_path)

cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    results = model(frame) # Make detections
    person_results = results.pred[0]  # 'person'

    if len(person_results) > 0:
        largest_box = max(person_results[:, 4])

        # Create mask
        mask = np.zeros_like(frame)
        for result in person_results:
            if result[4] != largest_box:
                x1, y1, x2, y2 = map(int, result[:4])
                mask[y1:y2, x1:x2] = 0  # masking

        # Get frame in the biggest bounding box
        largest_box_index = np.argmax(person_results[:, 4])
        x1, y1, x2, y2 = map(int, person_results[largest_box_index, :4])
        largest_box_frame = frame[y1:y2, x1:x2]
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # Draw a bounding box with colored green

        # Create the original frame by applying the mask to the largest bounding box
        result_frame = largest_box_frame + mask[y1:y2, x1:x2] 
        cv.imshow('Detect_only_Person', result_frame)

    if cv.waitKey(10) & 0xFF == ord('q'):# 'q' == break
        break

cap.release()
cv.destroyAllWindows()

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-10 Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AttributeError: 'NoneType' object has no attribute 'shape'

In [1]:
import cv2 as cv
import torch
import numpy as np
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

model_weights_path = './trained_model/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_weights_path)

cap = cv.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)  # Make detections

        person_results = results.pred[0]  # 'person'

        if len(person_results) > 0:
            # Find the largest bounding box
            largest_box_index = np.argmax(person_results[:, 4])
            x1, y1, x2, y2 = map(int, person_results[largest_box_index, :4])

            person_roi = frame[y1:y2, x1:x2] # 객체 바운딩 박스 안에 있는 부분만 추출

            # Recolor Feed
            person_roi = cv.cvtColor(person_roi, cv.COLOR_BGR2RGB)
            person_roi.flags.writeable = False

            # Make Detections
            results_pose = pose.process(person_roi)

            # Recolor image back to BGR for rendering
            person_roi.flags.writeable = True
            person_roi = cv.cvtColor(person_roi, cv.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results_pose.pose_landmarks.landmark
                mp_drawing.draw_landmarks(person_roi, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                          mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
            except:
                pass

            cv.imshow('Raw Webcam Feed', person_roi)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()


objc[39022]: Class CaptureDelegate is implemented in both /Users/min_leon/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x11f1865e0) and /Users/min_leon/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x130630860). One of the two will be used. Which one is undefined.
objc[39022]: Class CVWindow is implemented in both /Users/min_leon/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x11f186630) and /Users/min_leon/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x12031ca68). One of the two will be used. Which one is undefined.
objc[39022]: Class CVView is implemented in both /Users/min_leon/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x11f186658) and /Users/min_leon/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x12031ca90). One of the

In [41]:
import sys
import cv2
import glob
import os

image_path = "C:/Users/User/Desktop/AI_Exercise_Pose_Feedback-main/yolov5_onlyPerson/yolov5/runs/detect/exp4/crops/person/"
video_name = "output_video.mp4"

def image_load(image_path):

    img_array = []

    for filename in sorted(glob.glob(os.path.join(image_path, "*.jpg"))):
        img = cv2.imread(filename)
        img = cv2.resize(img, dsize=(720, 1000), interpolation=cv2.INTER_LINEAR)
        h, w, channels = img.shape
        size = (w, h)  # 너비와 높이 순으로 크기 설정
        img_array.append(img)

    return size, img_array

def video_generator(folder_path, file_name, size, fps, img_array):

    out = cv2.VideoWriter(filename=os.path.join(folder_path, file_name), fourcc=cv2.VideoWriter_fourcc(*'mp4v'), fps=fps, frameSize=size)

    for img in img_array:
        out.write(img)

    out.release()

def video_play(file_name):

    try:
        cap = cv2.VideoCapture(file_name)

        while (cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow("frame", gray)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    img_size, img_array = image_load(image_path)
    video_generator(folder_path=image_path, file_name=video_name, size=img_size, fps=30, img_array=img_array)
    video_play(file_name=video_name)
